<a href="https://colab.research.google.com/github/sudhir2016/Alexa/blob/master/Sentinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install sentinalsat, geopandas and rasterio.

In [ ]:
! pip install sentinelsat

In [ ]:
! pip install geopandas

In [ ]:
! pip install rasterio

Various imports.

In [4]:
from sentinelsat import SentinelAPI,read_geojson, geojson_to_wkt
import geopandas as gpd
import rasterio as rio
from rasterio.plot import show
from rasterio import mask
import numpy as np
import glob

User name and Password for Sentinal.

In [5]:
user= 'user name'

In [6]:
password= 'pass word'

Create sentinal query based on our plot coordinates,  required time window and cloud cover filter. Sentinal will provide list of availalble tiles called products.

In [7]:
api = SentinelAPI(user, password, 'https://scihub.copernicus.eu/dhus')

In [9]:
footprint = geojson_to_wkt(read_geojson('plot.geojson'))

In [10]:
products = api.query(footprint,
                     date = ('20200701', '20200831'),
                     platformname = 'Sentinel-2',
                     processinglevel = 'Level-2A',
                     cloudcoverpercentage = (0,100)
                    )

load products as geopandas dataframe.

In [ ]:
products_gdf = api.to_geodataframe(products)
print(products_gdf)

Obtain ID of most recent Sentinal dataset.

In [12]:
uuid=products_gdf['uuid']

In [12]:
#cloud=products_gdf['cloudcoverpercentage']
#print(cloud)

In [ ]:
uuid1=uuid[0]
print(uuid1)

Download most recent Sentinal data.

In [ ]:
api.download(uuid1)

Obtain filename of downloaded Sentilnal zip file.

In [ ]:
for name in glob.glob('/content/*.zip'):
  file=name
print(file)

Unzip Sentinal file.

In [ ]:
! unzip '$file'

Obtain the name to the Sentinal data directory.

In [ ]:
for name in glob.glob('/content/*.SAFE'):
  file1=name
print(file1)

Obtain the path of the Sentinal image data.

In [ ]:
for name in glob.glob(file1+'/GRANULE/*'):
  file2=name
print(file2)

Obtain the file names of B2(B),B3(G),B4(R) and B8(NIR) channel images.

In [18]:
file3=[]

In [ ]:
for name in glob.glob(file2+'/IMG_DATA/R10m/*'):
  file3.append(name)
file3.sort()
print(file3)

Read the B2, B3, B4 and B8 channel data.

In [20]:
b4=rio.open(file3[3])

In [21]:
b3=rio.open(file3[2])

In [22]:
b2=rio.open(file3[1])

In [23]:
b8=rio.open(file3[4])

Create RGB tiff file.

In [24]:
with rio.open('RGB.tiff','w',driver='Gtiff', width=b4.width, height=b4.height, 
              count=3,crs=b4.crs,transform=b4.transform, dtype=b4.dtypes[0]) as rgb:
    rgb.write(b2.read(1),1) 
    rgb.write(b3.read(1),2) 
    rgb.write(b4.read(1),3) 
    rgb.close()

Show RGB.tiff for the complete Sentinal tile.

In [ ]:
src = rio.open("RGB.tiff")
show(src)

Read B4(red) and B8(NIR) data and calculate NDVI.

In [26]:
red = b4.read()
nir = b8.read()

In [ ]:
ndvi = (nir.astype(float)-red.astype(float))/(nir+red)

In [ ]:
print(ndvi)

Create NDVI tiff file.

In [29]:
meta = b4.meta
meta.update(driver='GTiff')
meta.update(dtype=rio.float32)

with rio.open('NDVI.tiff', 'w', **meta) as dst:
    dst.write(ndvi.astype(rio.float32))

Show NDVI for complete Sentinal tile.

In [ ]:
src = rio.open("NDVI.tiff")
show(src)

Extract the Coordinate Reference System(crs) of the Sentinal data.

In [31]:
crs=src.crs
print(crs)

EPSG:32643


Convert plot coodinates to Sentinal crs.

In [32]:
plot=gpd.read_file('plot.geojson')

In [ ]:
plot1=plot.to_crs({'init': crs})

In [ ]:
geo=plot1.geometry
print(geo)

Extract plot RGB from Sentinal data.

In [35]:
with rio.open("RGB.tiff") as src:
    out_image, out_transform = rio.mask.mask(src,geo,crop=True)
    out_meta = src.meta.copy()
    out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})

Create RGB tiff file for plot and show.

In [36]:
with rio.open("RGB_plot.tiff", "w", **out_meta) as dest:
    dest.write(out_image)

In [ ]:
src = rio.open("RGB_plot.tiff")
show(src)

Extract plot NDVI.

In [38]:
with rio.open("NDVI.tiff") as src:
    out_image, out_transform = rio.mask.mask(src,geo,crop=True)
    out_meta = src.meta.copy()
    out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})

Create NDVI tiff file for plot and show.

In [39]:
with rio.open("NDVI_plot.tiff", "w", **out_meta) as dest:
    dest.write(out_image)

In [ ]:
src = rio.open("NDVI_plot.tiff")
show(src)

Read plot NDVI data, drop zero values and calculate mean NDVI.

In [41]:
data=src.read(1)

In [42]:
data1=data[data != 0]

In [ ]:
Mean_NDVI_Plot=np.sum(data1)/data1.size
print(Mean_NDVI_Plot)